In [ ]:
import cv2
from keras.models import model_from_json
import numpy as np
from IPython.display import display, clear_output
import PIL.Image
import time
import tensorflow as tf

# Ensure that Sequential is registered for deserialization
tf.keras.utils.get_custom_objects()['Sequential'] = tf.keras.Sequential

# Load the model architecture and weights
json_file = open("emotiondetector1.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector1.h5")

# Load the Haar cascade for face detection
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Define the labels
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Function to extract features
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Initialize the webcam
webcam = cv2.VideoCapture(0)

try:
    while True:
        # Read a frame from the webcam
        ret, frame = webcam.read()
        if not ret:
            break

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            # Extract the face region
            face = gray[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Preprocess the face image
            face = cv2.resize(face, (48, 48))
            img = extract_features(face)

            # Make prediction
            pred = model.predict(img)
            prediction_label = labels[pred.argmax()]

            # Display the label on the frame
            cv2.putText(frame, prediction_label, (x-10, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))

        # Display the frame in the notebook
        clear_output(wait=True)
        display(PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

        # Add a short delay
        time.sleep(0.1)

except KeyboardInterrupt:
    # Release the webcam when interrupted
    webcam.release()
    print("Webcam feed interrupted and stopped.")
